## Imports

In [3]:
import xarray as xr 
import numpy as np 
import numpy.ma as ma
import glob

import matplotlib.pyplot as plt
import pandas as pd
import os 
import copy
from scipy import signal

import matplotlib.pyplot as plt
import matplotlib as mpl
#import collections
import matplotlib.ticker as mticker
from matplotlib import ticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from matplotlib.colors import ListedColormap
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cmocean
import scipy as sp
import warnings
warnings.filterwarnings("ignore")
import os
import eofs.standard as Eof_st
from eofs.multivariate.standard import MultivariateEof
from eofs.xarray import Eof
from scipy.signal import butter, lfilter
from scipy.signal import convolve
import importlib
def reimport_module(module_name):
    """
    Reimports the specified Python module.
    
    Args:
    module_name (module): The module to be reimported, typically passed as a variable.
    
    Returns:
    module: The reloaded module.
    """
    try:
        # Reload the already imported module
        reloaded_module = importlib.reload(module_name)
        print(f"Module {module_name.__name__} reloaded successfully.")
        return reloaded_module
    except Exception as e:
        print(f"Failed to reload module: {e}")
import getpass
myname = getpass.getuser()

## Dask Baby

In [7]:
# ### ++++++ dask NCAR client: 
from distributed import Client
from dask_jobqueue import PBSCluster
project_num = 'P03010039' #what project key do you have? 

cluster = PBSCluster(account=project_num,walltime='12:00:00',cores=1, memory='40GB',shared_temp_directory='/glade/derecho/scratch/wchapman/tmp',queue='casper')
cluster.scale(jobs=40)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/diehard/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/diehard/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.110:34919,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/diehard/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Get File Paths

In [121]:
path2cesmPI = '/glade/campaign/cesm/development/cvcwg/cvwg/f.e21.F1850.f09_f09_mg17.DAILYSSTIC_CMIP6-piControl.001/atm/proc/tseries/day_1/'
SSTwc = '*.SST.*nc'
SSTfiles = sorted(glob.glob(os.path.join(path2cesmPI, SSTwc)))
TREFHTwc = '*.TREFHT.*nc'
TREFHTfiles = sorted(glob.glob(os.path.join(path2cesmPI, TREFHTwc)))

## Open Data

In [162]:
DSsst = xr.open_mfdataset(SSTfiles, parallel=True)
DStrefht = xr.open_mfdataset(TREFHTfiles, parallel=True).sel(lat=slice(25,65),lon=slice(200,250))

## Fix The Dates Cause CESM is a Nightmare

In [163]:
# Create the date range
date_range = pd.date_range(start='1700-01-01', end='2200-01-01')

# Filter out February 29th
filtered_date_range = date_range[~((date_range.month == 2) & (date_range.day == 29))]

# Display the filtered date range
filtered_date_range

DSsst['time'] = filtered_date_range
DSsst = DSsst['SST'].to_dataset(name='SST')
DStrefht['time'] = filtered_date_range
DStrefht = DStrefht['TREFHT'].to_dataset(name='TREFHT')

## Do a 60 day rolling mean:

In [164]:
#SST: 
DSmean_sst = DSsst.sel(time=slice('1700','2000')).mean('time').persist() #training data
DSmean_sst_anom = (DSsst-DSmean_sst).persist()

#TREFTHT: 
DSmean_trefht = DStrefht.sel(time=slice('1700','2000')).mean('time').persist() #training data
DStrefht = DStrefht['TREFHT'].to_dataset(name='TREHT')
DSmean_trefht_anom = (DStrefht-DSmean_trefht).drop(['time']).persist()
DSmean_trefht_anom_roll = DStrefht.rolling(time=60, min_periods=1).mean()

## Files out

In [165]:
dirout = f'/glade/derecho/scratch/{myname}/TOMSensoDS/'

# Check if the directory exists
if not os.path.exists(dirout):
    # Create the directory
    os.makedirs(dirout)
    print(f"Directory {dirout} created.")
else:
    print(f"Directory {dirout} already exists.")

DSmean_sst_anom.to_netcdf(os.path.join(dirout, 'SST_CESM2_PI.nc'))
print('...done saving SST...')

DSmean_trefht_anom_roll.to_netcdf(os.path.join(dirout, 'TREFHT_CESM2_PI.nc'))
print('...done saving TREFHT...')

Directory /glade/derecho/scratch/wchapman/TOMSensoDS/ already exists.
...done saving TREFHT...


In [168]:
print('...shutting down dask client...')
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

...shutting down dask client...
...shutdown client...


In [169]:
rm dask-worker*

rm: cannot remove 'dask-worker*': No such file or directory


In [ ]:
# Chunking files:
# DSmean_sst_anom = xr.open_dataset(os.path.join(dirout, 'SST_CESM2_PI.nc'))
# DSmean_trefht_anom_roll = xr.open_dataset(os.path.join(dirout, 'TREFHT_CESM2_PI.nc'))

# chunk_sizes = {'SST': {'time': 64, 'lat': 192, 'lon': 288},'TREHT':{'time': 64, 'lat': 42, 'lon': 41}}
# DSmean_sst_anom.SST.chunk(chunk_sizes['SST'])
# DSmean_trefht_anom_roll.TREHT.chunk(chunk_sizes['TREHT'])

# DSmean_sst_anom.to_netcdf(dirout+"SST_CESM2_PI_Chunked.nc", 
#                           encoding={'SST': {'chunksizes': [chunk_sizes['SST']['time'],
#                                                            chunk_sizes['SST']['lat'],
#                                                            chunk_sizes['SST']['lon']]}})

# DSmean_trefht_anom_roll.to_netcdf(dirout+"TREFHT_CESM2_PI_Chunked.nc", 
#                                   encoding={'TREHT': {'chunksizes': [chunk_sizes['TREHT']['time'],
#                                                                    chunk_sizes['TREHT']['lat'],
#                                                                    chunk_sizes['TREHT']['lon']]}})